### This is Example 4.3. Gambler’s Problem from Sutton's book.

A gambler has the opportunity to make bets on the outcomes of a sequence of coin flips. 
If the coin comes up heads, he wins as many dollars as he has staked on that flip; 
if it is tails, he loses his stake. The game ends when the gambler wins by reaching his goal of $100, 
or loses by running out of money. 

On each flip, the gambler must decide what portion of his capital to stake, in integer numbers of dollars. 
This problem can be formulated as an undiscounted, episodic, finite MDP. 

The state is the gambler’s capital, s ∈ {1, 2, . . . , 99}.
The actions are stakes, a ∈ {0, 1, . . . , min(s, 100 − s)}. 
The reward is zero on all transitions except those on which the gambler reaches his goal, when it is +1.

The state-value function then gives the probability of winning from each state. A policy is a mapping from levels of capital to stakes. The optimal policy maximizes the probability of reaching the goal. Let p_h denote the probability of the coin coming up heads. If p_h is known, then the entire problem is known and it can be solved, for instance, by value iteration.

In [1]:
import os, sys, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

if "../" not in sys.path:
  sys.path.append("../")


### Exercise 4.9 (programming)

Implement value iteration for the gambler’s problem and solve it for p_h = 0.25 and p_h = 0.55.



In [ ]:
def value_iteration_for_gamblers(p_h, theta=0.0001, discount_factor=1.0):
    """
    Args:
        p_h: Probability of the coin coming up heads
    """
    rewards = np.zeros(101)
    rewards[100] = 1
    V = np.zeros(101)
    
    def one_step_lookahead(s, V, rewards):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            s: The gambler’s capital. Integer.
            V: The vector that contains values at each state. 
            rewards: The reward vector.
        Returns:
            A vector containing the expected value of each action. 
            Its length equals to the number of actions.
        """
        A = np.zeros(101)

        # NOTE: Minimum bet must be 1 - cannot bet 0 (no point doing so anyways).
        # This is how much we are betting.
        action_space = np.arange(1, min(s, 100 - s) + 1)
        for action in action_space:
            # Expected value of reward.
            # p_h (win) -> expected reward value of rewards[state + action].
            # 1 - p_h (lose) -> expected reward value of rewards[state - action].

            # Don't really need discount factor since this problem is undiscounted.
            A[action] = p_h * (rewards[s + action] + V[s + action]*discount_factor) + (1 - p_h) * (rewards[s - action] + V[s - action]*discount_factor)
        return A
    
    while True:
        delta = 0
        # Cycle through all states/capital and find best action.
        for state in range(1, 100):
            actions = one_step_lookahead(state, V, rewards)
            best_action_reward = max(actions)

            # Store largest change in reward for this iteration to terminate when convergence.
            delta = max(delta, best_action_reward - V[state])
            V[state] = best_action_reward
        
        if (delta < theta): break
    
    # Build policy from values.
    policy = np.zeros(100)
    for capital in range(1, 100):
        action = one_step_lookahead(capital, V, rewards)
        best_action = np.argmax(action)
        policy[capital] = best_action

    return policy, V

In [35]:
policy, v = value_iteration_for_gamblers(0.25)

print("Optimized Policy:")
print(policy)
print("")

print("Optimized Value Function:")
print(v)
print("")

Optimized Policy:
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 12. 11. 15. 16. 17.
 18.  6. 20. 21.  3. 23. 24. 25.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.
 11. 12. 38. 11. 10.  9. 42.  7. 44.  5. 46. 47. 48. 49. 50.  1.  2.  3.
  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 11. 10.  9. 17.  7. 19.  5. 21.
 22. 23. 24. 25.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 12. 11.
 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.]

Optimized Value Function:
[0.00000000e+00 7.24792480e-05 2.89916992e-04 6.95257448e-04
 1.16010383e-03 1.76906586e-03 2.78102979e-03 4.03504074e-03
 4.66214120e-03 5.59997559e-03 7.08471239e-03 9.03964043e-03
 1.11241192e-02 1.56793594e-02 1.61464431e-02 1.69517994e-02
 1.86512806e-02 1.98249817e-02 2.24047303e-02 2.73845196e-02
 2.83388495e-02 3.04937363e-02 3.61633897e-02 3.84953022e-02
 4.44964767e-02 6.25000000e-02 6.27174377e-02 6.33700779e-02
 6.45857723e-02 6.59966059e-02 6.78135343e-02 7.08430894e-02
 7.46098323e-02 7.64884604e-02 7.93035477e-02 8.37541372e-02

In [36]:
# Plotting Final Policy (action stake) vs State (Capital)
fig = px.bar(x=range(100), y=policy, 
             labels={ 'x': 'Capital', 'y': 'Stake'})
fig.update_layout(
    bargap=0.0,
    paper_bgcolor = 'black',
    plot_bgcolor = 'black',
    font=dict(color='white'),
    title_font_color='white'
)

fig.update_traces(marker_line_color='black')
fig.show()

In [15]:
# Plotting Capital vs Final Policy

# Implement!
